Author: Jitender Singh Virk ||
Date created: 3 Jan 2018

In [ ]:
import math
import tf_utils
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
# dataset directory and files
#data_location = "C:\\Users\\Virksaab\\Desktop\\AI\\datasets_for_code_in_repos\\char74k_dataset\\"
data_location = "/home/virk/Desktop/AI/datasets_for_code_in_repos/char74k_dataset/"
TRAIN_FILE = "train.tfrecords"
VALIDATE_FILE = "validate.tfrecords"
TEST_FILE = "test.tfrecords"

In [ ]:
# Global constants describing the data set.
NUM_CLASSES = 62
IMAGE_SIZE = 64
IMAGE_PIXELS = IMAGE_SIZE * IMAGE_SIZE * 3
TRAIN_SET_SIZE = 8704
VALIDATE_SET_SIZE = 1792
TEST_SET_SIZE = 1792

In [ ]:
# HyperParameters for traning
neurons_in_layer = [IMAGE_PIXELS, 1984, 992, 496, 248, 124, NUM_CLASSES]
num_epochs = 20
bs = 1792
train_batch_size = 8704
val_batch_size = bs
test_batch_size = bs
iterations = 1000
learning_rate = 0.0001 # Decaying if start_
L2_beta = 0.01

In [ ]:
def build_graph(batch_size, is_training):
    # READ DATA
    # Read training data from tfrecords file
    train_x_batch, train_y_batch = tf_utils.read_images_from_tfrecords_in_shuffle_batch(data_location+TRAIN_FILE, image_shape=IMAGE_PIXELS,
                                                                                        batch_size=train_batch_size, num_epochs=num_epochs, shuffle=True)
    train_y_batch = tf.one_hot(train_y_batch, NUM_CLASSES)
    #train_x_batch, train_y_batch = tf.transpose(train_x_batch), tf.transpose(train_y_batch)

    # Read validation data from tfrecords file
    val_x_batch, val_y_batch = tf_utils.read_images_from_tfrecords_in_shuffle_batch(data_location+VALIDATE_FILE, image_shape=IMAGE_PIXELS,
                                                                                    batch_size=val_batch_size, num_epochs=1, shuffle=True)
    val_y_batch = tf.one_hot(val_y_batch, NUM_CLASSES)
    #val_x_batch, val_y_batch = tf.transpose(val_x_batch), tf.transpose(val_y_batch)

    # Read testing data from tfrecords file
    test_x_batch, test_y_batch = tf_utils.read_images_from_tfrecords_in_shuffle_batch(data_location+TEST_FILE, image_shape=IMAGE_PIXELS,
                                                                                    batch_size=test_batch_size, num_epochs=1, shuffle=True)
    test_y_batch = tf.one_hot(test_y_batch, NUM_CLASSES)
    #test_x_batch, test_y_batch = tf.transpose(test_x_batch), tf.transpose(test_y_batch)
    
    # PLACEHOLDERS
    X = tf.placeholder(tf.float32, shape=[batch_size, IMAGE_PIXELS])
    Y = tf.placeholder(tf.float32, shape=[batch_size, NUM_CLASSES])
    keep_prob = tf.placeholder(tf.float32)
    # LAYERS
    ## Layer-1
    w1 = tf.get_variable("w1", [neurons_in_layer[1], neurons_in_layer[0]], initializer=tf.contrib.layers.xavier_initializer())
    z1 = tf.matmul(w1, tf.transpose(X))
    z1_bn, bn_cache1 = tf_utils.batch_norm_wrapper(1, z1, is_training=is_training)
    a1 = tf.nn.relu(z1_bn)
    a1_dropout = tf.nn.dropout(a1, keep_prob)
    ## Layer-2
    w2 = tf.get_variable("w2", [neurons_in_layer[2], neurons_in_layer[1]], initializer=tf.contrib.layers.xavier_initializer())
    z2 = tf.matmul(w2, a1_dropout)
    z2_bn, bn_cache2 = tf_utils.batch_norm_wrapper(2, z2, is_training=is_training)
    a2 = tf.nn.relu(z2_bn)
    a2_dropout = tf.nn.dropout(a2, keep_prob)
    ## Layer-3
    w3 = tf.get_variable("w3", [neurons_in_layer[3], neurons_in_layer[2]], initializer=tf.contrib.layers.xavier_initializer())
    z3 = tf.matmul(w3, a2_dropout)
    z3_bn, bn_cache3 = tf_utils.batch_norm_wrapper(3, z3, is_training=is_training)
    a3 = tf.nn.relu(z3_bn)
    a3_dropout = tf.nn.dropout(a3, keep_prob)
    ## Layer-4
    w4 = tf.get_variable("w4", [neurons_in_layer[4], neurons_in_layer[3]], initializer=tf.contrib.layers.xavier_initializer())
    z4 = tf.matmul(w4, a3_dropout)
    z4_bn, bn_cache4 = tf_utils.batch_norm_wrapper(4, z4, is_training=is_training)
    a4 = tf.nn.relu(z4_bn)
    a4_dropout = tf.nn.dropout(a4, keep_prob)
    ## Layer-5
    w5 = tf.get_variable("w5", [neurons_in_layer[5], neurons_in_layer[4]], initializer=tf.contrib.layers.xavier_initializer())
    z5 = tf.matmul(w5, a4_dropout)
    z5_bn, bn_cache5 = tf_utils.batch_norm_wrapper(5, z5, is_training=is_training)
    a5 = tf.nn.relu(z5_bn)
    a5_dropout = tf.nn.dropout(a5, keep_prob)
    ## Layer-6
    w6 = tf.get_variable("w6", [neurons_in_layer[6], neurons_in_layer[5]], initializer=tf.contrib.layers.xavier_initializer())
    z6 = tf.matmul(w6, a5_dropout)
    z6_bn, bn_cache6 = tf_utils.batch_norm_wrapper(6, z6, is_training=is_training)

    Yhat = tf.transpose(z6_bn)
    
    # Compute loss
    loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=Yhat, labels=Y))
    # Loss function with L2 Regularization with decaying learning rate
    regularizers = tf.nn.l2_loss(w1) + tf.nn.l2_loss(w2) + tf.nn.l2_loss(w3) + tf.nn.l2_loss(w4) + tf.nn.l2_loss(w5) + tf.nn.l2_loss(w6)           
    loss = tf.reduce_mean(loss + L2_beta * regularizers)
    
    # Optimizer
    optimizer = tf.train.AdamOptimizer(learning_rate).minimize(loss)
    
    # save your variables for later use
    saver = tf.train.Saver()
    
    if is_training:
        return (train_x_batch, train_y_batch), (X, Y, keep_prob), optimizer, loss, Yhat, saver, bn_cache1
    else:
        return (val_x_batch, val_y_batch, test_x_batch, test_y_batch), (X, Y, keep_prob), Yhat, saver


In [ ]:
#Build training graph, train and save the trained model
tf.reset_default_graph()
(train_x_batch, train_y_batch), (X, Y, keep_prob), optimizer, loss, Yhat, saver, bn_cache = build_graph(train_batch_size, is_training=True)

# Start Session
config = tf.ConfigProto()
config.gpu_options.allow_growth = True
sess = tf.Session(config=config)

# Number of batches in training data   
num_batches = math.ceil(TRAIN_SET_SIZE / train_batch_size)

# Initialiize Variables
init_op = tf.group(tf.global_variables_initializer(), tf.local_variables_initializer())
sess.run(init_op)

# Set coordinator and start queue runner for training set
coord = tf.train.Coordinator()
threads = tf.train.start_queue_runners(sess=sess, coord=coord)  
 
# TRAINING
for epoch in range(1, num_epochs+1):
    print("EPOCH {}/{}".format(epoch, num_epochs), "-"*40)
    for batch_no in range(1, (num_batches+1)):
        try:
            train_X, train_Y = sess.run([train_x_batch, train_y_batch])
            for iteration in range(1, iterations+1):
                print(iteration)
                _, batch_loss = sess.run([optimizer, loss], {X:train_X, Y:train_Y, keep_prob:0.5})
            print("loss of batch no. {}/{} after optimizing with {} iters is {}".format(batch_no, num_batches, iterations, batch_loss))
    
        except tf.errors.OutOfRangeError:
            print('TRAIN batch limit reached')
            
        #print(sess.run([tf.argmax(tf.nn.softmax(Yhat), 1), tf.argmax(Y, 1)], {X:train_X, Y:train_Y, keep_prob:1.0}))
        acc = tf.reduce_mean(tf.cast(tf.equal(tf.argmax(tf.nn.softmax(Yhat), 1), tf.argmax(Y, 1)), "float")) * 100.
        print("Train accuracy(per batch) =", sess.run(acc, {X:train_X, Y:train_Y, keep_prob:1.0}))
    
sc, bt, pm, pv = sess.run(bn_cache)
#print(sc.shape, bt.shape, pm.shape, pv.shape)

# Save your training
save_path = saver.save(sess, "./char74k_training/training")

# Wait for training threads to finish.
coord.request_stop()
coord.join(threads)
# Don't forget to close session
sess.close()  

In [ ]:
#Build Validate and testing graph
tf.reset_default_graph()
(val_x_batch, val_y_batch, test_x_batch, test_y_batch), (X, Y, keep_prob), Yhat, saver = build_graph(val_batch_size, is_training=False)

# Start Session
config = tf.ConfigProto()
config.gpu_options.allow_growth = True
sess = tf.Session(config=config)

# Number of batches in validate data   
num_val_batches = math.ceil(VALIDATE_SET_SIZE / val_batch_size)
# Number of batches in testing data
num_test_batches = math.ceil(TEST_SET_SIZE / test_batch_size)

# Initialiize Variables
init_op = tf.group(tf.global_variables_initializer(), tf.local_variables_initializer())
sess.run(init_op)

# Restore saved training variables
saver.restore(sess, "./char74k_training/training")

# Set coordinator and start queue runner for training set
coord = tf.train.Coordinator()
threads = tf.train.start_queue_runners(sess=sess, coord=coord) 

# VALIDATE
for batch_no in range(1, (num_val_batches+1)):
        try:
            val_X, val_Y = sess.run([val_x_batch, val_y_batch])

             # Calculate accuracy on the validate set
            val_accuracy = tf.reduce_mean(tf.cast(tf.equal(tf.argmax(tf.nn.softmax(Yhat), 1), tf.argmax(Y, 1)), "float")) * 100.
            print("Validate Accuracy of batch {} is {}".format(batch_no, sess.run(val_accuracy, {X: val_X, Y: val_Y, keep_prob:1.0})))
        except tf.errors.OutOfRangeError:
            print('VALIDATE batch limit reached')

#print(sess.run([tf.argmax(tf.nn.softmax(Yhat), 1), tf.argmax(Y, 1)], {X: val_X, Y: val_Y, keep_prob:1.0}))            
# TESTING
for batch_no in range(1, (num_test_batches+1)):
        try:
            test_X, test_Y = sess.run([test_x_batch, test_y_batch])

             # Calculate accuracy on the test set
            test_accuracy = tf.reduce_mean(tf.cast(tf.equal(tf.argmax(tf.nn.softmax(Yhat), 1), tf.argmax(Y, 1)), "float")) * 100.
            print("Testing Accuracy of batch {} is {}".format(batch_no, sess.run(test_accuracy, {X: test_X, Y: test_Y, keep_prob:1.0})))
            
        except tf.errors.OutOfRangeError:
            print('TEST batch limit reached')

            
# Wait for training threads to finish.
coord.request_stop()
coord.join(threads)
# Don't forget to close session
sess.close() 

Copyright © 2018, Jitender Singh Virk

Licensed under the Apache License, Version 2.0 (the "License");
you may not use this file except in compliance with the License.
You may obtain a copy of the License at

    http://www.apache.org/licenses/LICENSE-2.0

Unless required by applicable law or agreed to in writing, software
distributed under the License is distributed on an "AS IS" BASIS,
WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
See the License for the specific language governing permissions and
limitations under the License.